In [0]:
from google.colab import drive
drive.mount('/content/drive/')

# Goole drive push data and models

In [0]:
!unzip drive/My\ Drive/ammi-2020-convnets.zip \

# !mkdir models
# !cp -r drive/My\ Drive/data/models/* models/.


# !cp -r models/* drive/My\ Drive/data/models/.

In [0]:
!pip install timm  -q
!pip install pretrainedmodels -q
!pip install optuna -q

In [0]:
from __future__ import print_function
from __future__ import division

import time
import os
import copy
import glob

import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models



import numpy as np
import pandas as pd
import datetime as dt


import pretrainedmodels
import timm

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image




print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

%matplotlib inline

# Dataset

In [0]:
data_path = "train/train/"
test_path = "test/test/0"
extraimage_path = "extraimages/extraimages"

In [0]:
print('Train set:')
class_distrbution = {}
for cls in os.listdir(data_path):
    print('{}:{}'.format(cls, len(os.listdir(os.path.join(data_path, cls)))))
    class_distrbution[cls] =  len(os.listdir(os.path.join(data_path, cls)))
im = Image.open(data_path+'/cgm/train-cgm-738.jpg')
print(im.size)
class_distrbution

In [0]:
# Transformations for both the training and testing data
mean=[0.4543, 0.5137, 0.3240]
std=[0.1949, 0.1977, 0.1661]


train_transforms = transforms.Compose([transforms.RandomResizedCrop(320),
                                       transforms.RandomRotation(30),#448, 299, 224, 331
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])

test_transforms = transforms.Compose([ transforms.Resize(320),
                                       transforms.CenterCrop(320),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])

# normalize = transforms.Normalize(mean=mean, std=std)

In [0]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform

        files = []
        class_names = {}
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                files.append([i, className, fileName])

                name = str(i)+'-'+className
                if name not in class_names:
                    class_names[name] = 1
                else:
                    class_names[name] += 1
        self.file_list = files
        print(class_names)
        files = None

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(im)
        
# #         return im.view(3, 448, 448), classCategory
#         return im.view(3, 224, 224), classCategory
# #         return im.view(3, 299, 299), classCategory
        return im.view(3, 320, 320), classCategory   # NASNetLarge 331x331

In [0]:
train_data = CassavaDataset(data_path, transform=train_transforms)

test_data = CassavaDataset(test_path, transform=test_transforms)

extraimage_data = CassavaDataset(extraimage_path, transform=train_transforms) #maybe need an other trasforms, I had to change the dataset structure :)

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

validation_split = 0.2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]


# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=32,
                                             sampler=valid_sampler)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [0]:
def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5     # unnormalize
    npimg = img.cpu().numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [0]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(labels)
img_grid = torchvision.utils.make_grid(images)

matplotlib_imshow(img_grid, one_channel=False)

# Models

In [0]:
class Additional(nn.Module):
    def __init__(self, modelA,in_features,nb_classes=5,drop=0.2):
        super(Additional, self).__init__()
        self.modelA = modelA
        # Remove last linear layer
        self.modelA.classifier = nn.Identity()
        
        for p in self.modelA.parameters():
            p.requires_grad = False
        
        # Create new classifier
        self.fc_1 = nn.Linear(in_features,256)
        self.fc_2 = nn.Linear(256,  512)
        self.fc_out = nn.Linear( 512, nb_classes)
        
        #Dropout
        self.dropout = nn.Dropout(p=drop)
        
    def forward(self, x):
        #model
        x = self.modelA(x.clone())  
        x = x.view(x.size(0), -1)
        
        #FC
        x  = self.dropout(F.relu(self.fc_1(x)))
        x = self.dropout(F.relu(self.fc_2(x)))
        x = self.fc_out(x)
        
        return x

In [0]:
# this function is used during training process, to calculation the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [0]:
total_loss_train, total_acc_train = [],[]
def train(train_loader, model, criterion, optimizer, epoch,device):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        images, labels = data
        N = images.size(0)
#         print('image shape:',images.size(0), 'label shape',labels.size(0))
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
    return train_loss.avg, train_acc.avg

In [0]:
def validate(val_loader, model, criterion, optimizer, epoch,device):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)

            val_loss.update(criterion(outputs, labels).item())
            
    return val_loss.avg, val_acc.avg

In [0]:
def test(test_loader,train_loader, model):
    model.eval()
    pred = []
    _class_labels = np.array(train_loader.dataset.classes)
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            images, _ = data
            images = Variable(images).to(device)
    
            outputs = model(images)
    
            prediction = outputs.data.cpu().numpy().argmax()
            
            _predicted_class_labels = _class_labels[prediction]
            
            pred.append(_predicted_class_labels)

    return pred

# Find the right Hyper parameter

In [0]:
def objective(trial):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    drop  = trial.suggest_loguniform('drop', 0.1, 0.5)

    model = timm.create_model('efficientnet_b3a', pretrained=True)
    model = model.to(device)
    model_name = 'efficientnet_b3a'

    model = Additional(model,model.classifier.in_features,drop=drop)
    model = model.to(device)
    
    # class_weights = [class_distrbution[i] for i in train_loader.dataset.classes]
    # class_weights_normalized = [max(class_weights)/i for i in class_weights]

    # class_weights_normalized,torch.Tensor(class_weights_normalized)

    # weights = torch.Tensor(class_weights_normalized)
    # weights = weights.to(device)

    # criterion = nn.CrossEntropyLoss(weights)
    criterion = nn.CrossEntropyLoss()
    
    lr  = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    optim_ = trial.suggest_categorical('optim_',[optim.Adam])
    momentum = trial.suggest_uniform('momentum', 0.4, 0.99)
    optimizer = optim_(model.parameters(), lr=lr)
    
    epoch_num = 10
    best_val_acc = 0.85
    total_loss_val, total_acc_val = [],[]
    for epoch in range(1, epoch_num+1):
        loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch,device)
        loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, epoch,device)
        total_loss_val.append(loss_val)
        total_acc_val.append(acc_val)
        if acc_val > best_val_acc:
            best_val_acc = acc_val
            torch.save(model.state_dict(), model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt')
        print('*****************************************************')
        print('best record: [epoch %d], [Train loss %.5f], [Train acc %.5f], [val loss %.5f], [val acc %.5f]' % (epoch,loss_train,acc_train, loss_val, acc_val))
        print('*****************************************************')
            
     # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
            
            
    return acc_val

In [0]:
import optuna

sampler = optuna.samplers.TPESampler()
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(func=objective, n_trials=20)

# Train on the best ayperparameter

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
drop  = 0.307

model = timm.create_model('efficientnet_b3a', pretrained=True)
model = model.to(device)
model_name = 'efficientnet_b3a'

model = Additional(model,model.classifier.in_features,drop=drop)
model = model.to(device)

# class_weights = [class_distrbution[i] for i in train_loader.dataset.classes]
# class_weights_normalized = [max(class_weights)/i for i in class_weights]

# class_weights_normalized,torch.Tensor(class_weights_normalized)

# weights = torch.Tensor(class_weights_normalized)
# weights = weights.to(device)

# criterion = nn.CrossEntropyLoss(weights)
criterion = nn.CrossEntropyLoss()


lr  = 0.00031
# optim_ = trial.suggest_categorical('optim_',[optim.Adam])
optimizer = optim.Adam(model.parameters(), lr=lr)

epoch_num = 25
best_val_acc = 0.85
total_loss_val, total_acc_val = [],[]
for epoch in range(1, epoch_num+1):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch,device)
    loss_val, acc_val = validate(valid_loader, model, criterion, optimizer, epoch,device)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        torch.save(model.state_dict(), model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt')
        print('Model saved :{}'.format(model_name+'freeze_'+str(best_val_acc)[:4]+'.ckpt'))
    print('*****************************************************')
    print('[epoch %d], [Train loss %.5f], [Train acc %.5f], [val loss %.5f], [val acc %.5f]' % (epoch,loss_train,acc_train, loss_val, acc_val))
    print('*****************************************************')
        

## Sumission

In [0]:
class_names = train_loader.dataset.classes

In [0]:
def process_image(image_dir):
    # Process a PIL image for use in a PyTorch model
    # tensor.numpy().transpose(1, 2, 0)
    image = Image.open(image_dir)
    preprocess = transforms.Compose([ transforms.Resize(224),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                      transforms.Normalize(mean=mean,std=std)])
    image = preprocess(image)
    # Convert 2D image to 1D vector
    image = np.expand_dims(image, 0)
    image = torch.from_numpy(image)
    inputs = image.to(device)
    return inputs

In [0]:
# Using our model to predict the label
def predict(image, model):
    # Pass the image through our model
    output = model(image)
    # Reverse the log function in our output
    output = torch.exp(output)
    # Get the top predicted class, and the output percentage for
    # that class
    probs, classes = output.topk(1, dim=1)
    return probs.item(), classes.item()

In [0]:
test_directory = "./data/test/test/0"
predictions, test_image_fileName = [], []
try:
    test_images = listdir(test_directory)
    for images in test_images:
        test_image_fileName.append(images)
        image = process_image(f'{test_directory}/{images}')
        top_prob, top_class = predict(image, model)
        predictions.append(class_names[top_class])
except Exception as e:
    print(e)

In [0]:
print("[INFO] Creating pandas dataframe")
submission_data = {"Category":predictions,"Id":test_image_fileName,}
submission_data_frame = pd.DataFrame(submission_data)

In [0]:
submission_data_frame.head()

In [0]:
submission_data_frame.to_csv('submission'+model_name+'_freeze_86_flip.csv', header=True, index=False)